In [357]:
import pandas as pd
import numpy as np
import re

In [358]:
def remove_semicolons_front_back(value):
    if value != None and len(value) > 0:
        value = value.strip()
        while value[0]== ';':
            value = value[1:]
        while value[-1] == ';':
            value = value[:-1]
    return value
def replace_semicolons(value):
    value = value.replace(';',',')
    return value

def clean_bus_values(value):
    value = value.replace('s:','')
    value = value.replace(':','')
    value = value.replace('and',',')
    value = value.replace(', ,',',')
    value = value.replace('\n','')
    return value
    
def segment_location(location_string):
    #Column Order
    # Location, Phone Number, Fax Number, web-site, email-address.
    location_segments = location_string.split(';')
    #print(len(location_segments))
    #Address exist in all locations.
    value = location_segments[0].strip()+ ';' 
    
    count = 1
    if count < len(location_segments):
        if(location_segments[count].find('http') != -1 or location_segments[count].find('www') != -1):
            website_location = location_segments[count].find('http')
            if website_location == -1:
                website_location = location_segments[count].find('www')
            #print(website_location)
            value = value + str(location_segments[count][:website_location]) + ';;' + str(location_segments[count][website_location:])
            count = count + 3
        elif location_segments[count].find('Phone and Fax:') != -1:
            phone_string = location_segments[count].replace('Phone and Fax:', '') 
            value = value + phone_string + ';' + phone_string
            count = count + 2
        else:
            value = value + location_segments[1].strip()+';'
            count = count + 1
    
    #Check for fax
    #print(value)
    if count < len(location_segments) and location_segments[count].find('Fax:')!= -1:
        value = value + location_segments[count].replace('Fax:','').strip() + ';'
        count = count+1
    else:
        value = value + ';'
    #print(value)
    #Check for website.
    if count < len(location_segments) and (location_segments[count].find('http') != -1 
                                           or location_segments[count].find('www') != -1):
        if location_segments[count].find('@') != -1:
            value = value + " ".join(location_segments[count].split()).strip().replace(' ',';')
            count = count + 2
        else:
            value = value + location_segments[count].strip() + ';'
            count = count+1
    else:
        value = value + ';'
    #print(value)
    #Check for email-address
    if count < len(location_segments) and location_segments[count].find('@') != -1:
        value = value + location_segments[count].strip() + ';'
        count = count+1
    else:
        value = value + ';'
    #Remove the last semicolon.    
    while value.count(';') >= 5 and value[-1] == ';':
            value = value[:-1]
    #print(value)
    return value

In [359]:
#Read the json file.
clinic_df = pd.read_json('clinic_data_final.json', orient='split')

In [360]:
#Clean Data.
#1. Remove leading and trailing semicolons from all columns. 
#2. Devide Location into Location, Phone Number, Fax, website, and email.
#3. Remove s: and 'and' from Bus Routes.
#4. For remaining semicolons, replace them with commas.
clinic_df =clinic_df.applymap(remove_semicolons_front_back)
clinic_df['Bus Route'] = clinic_df['Bus Route'].apply(clean_bus_values)
clinic_df['Location'] = clinic_df['Location'].apply(segment_location)
clinic_df[['Address','Phone Number','Fax', 'Website', 'Email Address']] = clinic_df['Location'].str.split(';',expand=True)
clinic_df = clinic_df.applymap(replace_semicolons)

In [361]:
#Remove Location column and rearrange columns.
clinic_df = clinic_df[['Clinic', 'Address', 'Phone Number', 'Fax', 'Website', 'Email Address', 'Bus Route', 'Cross Streets', 'Hours', 'Language', 'Patient Eligibility', 'Special Populations Served', 'Payer Population Served', 'Payment Practices', 'Required Documents', 'Service Description', 'Clinical Services', 'Routine Services', 'Clinic Type']]

In [362]:
clinic_df

,Clinic,Address,Phone Number,Fax,Website,Email Address,Bus Route,Cross Streets,Hours,Language,Patient Eligibility,Special Populations Served,Payer Population Served,Payment Practices,Required Documents,Service Description,Clinical Services,Routine Services,Clinic Type
0,"AIDS Resource Center of Wisconsin,ARCW Medical...","820 N Plankinton Avenue, Milwaukee, WI 53203",(414) 223-6800,,www.arcw.org,,"14, 15, 19, 30, 57, GoldLine",N Plankinton Avenue and W Wells Street,"Monday – Friday 8:00 am - 4:00 pm, Call to sch...","English, Spanish, Language Line","All ages, Clinic will assist with insurance ...",All individuals diagnosed with HIV,"Commercial Insurance, Medicaid / ,BadgerCare, ...",Clinic bills 3rd party payer,Proof of HIV,The AIDS Resource Center of Wisconsin is home ...,"Behavioral Health Services ,(for patients wi...","(for patients with HIV), Flu Shots, HIV, Immu...",STD/HIV
1,Angel of Hope Clinic,"209 W Orchard Street, Milwaukee, WI 53204",(414) 647-7466,(414) 527-7153,,,"15, 56, GreenLine",S 2nd Street and W Orchard Street,"Monday – Wednesday 8:30 am – 4:30 pm, Thursday...","English, Spanish, Language Line","Ages 6 months and older, Clinic will assist ...",Homeless,"Charity Program, Commercial Insur-,ance, Medic...",Sliding fee scale,"Photo ID, proof of income, insurance card",The Angel of Hope Clinic provides a full range...,"Health Insurance ,Navigation Services, Labo...","Diabetes, Dietary/Nutrition Services, Flu Sho...","Medical ,Free or Charitable Clinic"
2,"Ascension Columbia St. Mary’s ,Ebenezer Healt...","3132 N Dr Martin Luther King Drive, Milwaukee,...",(414) 793-0328,,www.columbia-stmarys.org/Ebenezer-Health-Resou...,julia.means@ascension.org,"19, 60",N Dr Martin Luther King Drive and W Auer Avenue,"Wednesday and Thursday 9:00 am – 4:30 pm, Walk...","English, Language Line","All ages, Clinic does not provide primary car...","Pregnant Women, People with ,chronic disease s...","Commercial Insurance, Medicaid / ,BadgerCare, ...",Completely free to all users,Call to inquire,Ascension Columbia St. Mary’s Ebenezer Health ...,,"Health Information ,and Referral","Medical ,Free or Charitable Clinic"
3,"Ascension Columbia St. Mary’s ,Family Health C...","1121 E. North Avenue, Milwaukee, WI 53212",(414) 267-6500,(414) 267-3893,https://ascension.org,,"14, 21",N Humboldt Boulevard and E North Avenue,"Monday 9:00am – 5:00pm, Tuesday and Wednesday ...","English, Spanish, Language Line","All ages, Clinic will assist with insurance e...",Pregnant Women,"Commercial Insurance, Medicaid / ,BadgerCare, ...",Clinic bills third party payer,"Photo ID, insurance card","Provides primary care services to all ages, fr...","Behavioral Health Services, Therapy, Dental ...","Dementia Screening, Diabetes Education and ,...","Medical ,Free or Charitable Clinic"
4,Ascension Seton Dental Clinic,"1730 S 13th Street, Milwaukee, WI 53204",(414) 383-3220,(414) 383-3363,www.columbia-stmarys.org/Ann_Seton_Dental,,"17, 19, 54",S 13th Street and W Forest Home Avenue south ...,"Monday – Thursday 7:30 am – 4:00 pm, ,Friday ...","English, Spanish","All ages, Clinic will assist with insurance e...","Homeless, Pregnant Women","Medicaid / BadgerCare, Uninsured",Clinic bills 3rd party payer,"Proof of income, proof of address, photo ID",The clinic provides emergency dental services ...,"Dental Health Services, Health Insurance ,N...",,Dental
5,Ascension St. Ben’s Clinic,"1027 N 9th Street, Milwaukee, WI 53233",(414) 765-0606,(414) 765-0226,www.columbia-stmarys.org/St_Bens_Clinic,,"12, 31, 33, 80",N 9th and W State Streets,"Monday-Thursday 9:00 am – 4:00 pm, ,Friday 9:...","English, clinic has access to translators thro...","Adults 18 – 65, Clinic will assist with insur...",Homeless,"Medicaid / BadgerCare, Medicare, ,Uninsured","Sliding fee scale, clinic bills 3rd party payer",Call to inquire,Provides primary health care services to men a...,"Dental Service Referrals, Medication Service...","Diabetes, Flu Shots, Immunizations ,other t...","Medical ,Free or Ch

In [365]:
clinic_df.to_csv('Clean_Clinic_Data.csv')

In [366]:
clinic_df.to_json('Clean_Clinic_Data.json')